<a href="https://colab.research.google.com/github/YannC97/Yann-s-Coding-Projects/blob/Codes/Asset%20Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
Suppose there is a single agent with β = 0.95,
v(c) = log(c), there are two states
which are iid with equal probabilities π1 = π2 = 0.5 and the firm’s production
function is
f(k, 1) = 0.9k
0.3 + 0.9k, f(k, 2) = 1.1k
0.3 + 0.9k

Discretize the possible capital values to 50 points. Use value function iteration
to compute the policy functions (one for each shock) for consumption and investment.
Plot these functions. Now use 500 points for admissible capital levels
and redo the exercise.

b)
- Redo the same exercise with β = 0.99 and with v(c) = −c^−4
- Also redo the exercise with β = 0.95, log-utility but with persistent shocks.
Assume that π(1, 1) = π(2, 2) = 0.95.

For each case, compute the standard deviations of investment, consumption and output.
Also compute the average return to capital and the average risk-free rate.
Note that there is no bond in the setup of the problem, but with one
agent, adding a bond does not change anything and hence you can read off the
risk-free rate from your solutions using the Euler equation.'''

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.interpolate import CubicSpline

In [ ]:
# Parameters
alpha = 0.3 # Capital share in output
beta = 0.95 # Discount factor #later on Beta =
delta = 0.1 # Depreciation rate
A_low, A_high = 0.9, 1.1 # Productivity states
pi = np.array([0.5, 0.5]) # Transition probabilities (i.i.d.)
k_min, k_max, num_k = 0.1, 10, 50 # Capital grid
k_grid = np.linspace(k_min, k_max, num_k)
max_iter = 500
tol = 1e-6

In [ ]:


# Production function f(k, A)
def f(k, A):
return A * k**alpha + (1 - delta) * k

# Utility function u(c) = -c^(-4)
def u(c):
#return -(np.maximum(c, 1e-8)) ** (-4) # Avoid singularity at c = 0
return np.log(c)

# Consumption function ensuring positivity
def c(k, A, k_prime):
return np.maximum(f(k, A) - k_prime, 1e-8) # Ensure consumption is positive

# Marginal utility function
def mu_c(c):
return 4 * c**(-5)

# FOC for root-finding
def belman_foc(k_prime, k, A, V_prime, z):
return -mu_c(c(k, A, k_prime)) + beta * V_prime[z](k_prime)

# Howard's Policy Iteration
def policy_iteration(max_iter=500, tol=1e-6):
policy = np.full((num_k, 2), 0.5 * k_grid[:, None]) # Initial policy guess
V = np.zeros((num_k, 2)) # Initial value function
for iteration in range(max_iter):

# Step 1: Policy Evaluation
V_new = np.zeros_like(V)
V_splines = [CubicSpline(k_grid, V[:, z], bc_type='natural') for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
k_next = policy[i, z]
V_new[i, z] = u(c(k, A, k_next)) + beta * np.dot(pi, [V_splines[0](k_next), V_splines[1](k_next)])

# Check convergence
V_converged = np.max(np.abs(V_new - V)) < tol
V = V_new # Update value function

# Step 2: Policy Improvement
policy_stable = True
V_prime_splines = [CubicSpline(k_grid, np.gradient(V[:, z], k_grid), bc_type='natural') for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
solution = root(belman_foc, x0=policy[i, z], args=(k, A, V_prime_splines, z))
k_opt = solution.x[0] if solution.success and solution.x[0] > 0 else max(1e-6, f(k, A) * 0.5)
if np.abs(k_opt - policy[i, z]) > tol:
policy_stable = False
policy[i, z] = k_opt
if policy_stable and V_converged:
break
return V, policy

# Compute solution
V_policy, policy_policy = policy_iteration()
#3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
#https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 1/8
#Analytical Steady-State Capital: 4.1870
#Numerical Steady-State Capital (Low A): 2.1402
#Numerical Steady-State Capital (High A): 6.8648

# Plot Value Function
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_grid, V_policy[:, 0], 'b--', label='Value Function (Low A)')
plt.plot(k_grid, V_policy[:, 1], 'orange', label='Value Function (High A)')
plt.xlabel('Capital')
plt.ylabel('Value')
plt.legend()
plt.title('Value Function')

# Plot Policy Function
plt.subplot(1, 2, 2)
plt.plot(k_grid, policy_policy[:, 0], 'r--', label='Policy Function (Low A)')
plt.plot(k_grid, policy_policy[:, 1], 'b', label='Policy Function (High A)')
plt.xlabel('Capital Today')
plt.ylabel('Capital Tomorrow')
plt.legend()
plt.title('Policy Function')
plt.tight_layout()
plt.show()

# Compare steady-state values
k_star_numerical_low = policy_policy[np.argmin(np.abs(policy_policy[:, 0] - k_grid)), 0]
k_star_numerical_high = policy_policy[np.argmin(np.abs(policy_policy[:, 1] - k_grid)), 1]
k_star_analytical = ((1 - beta * (1 - delta)) / (beta * alpha)) ** (1 / (alpha - 1))
print(f"Analytical Steady-State Capital: {k_star_analytical:.4f}")
print(f"Numerical Steady-State Capital (Low A): {k_star_numerical_low:.4f}")
print(f"Numerical Steady-State Capital (High A): {k_star_numerical_high:.4f}")



In [ ]:
'''

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.interpolate import CubicSpline
# Parameters
alpha = 0.3 # Capital share in output
beta = 0.95 # Discount factor
delta = 0.1 # Depreciation rate
A_low, A_high = 0.9, 1.1 # Productivity states
pi = np.array([0.5, 0.5]) # Transition probabilities (i.i.d.)
k_min, k_max, num_k = 0.1, 10, 200 # Capital grid
k_grid = np.linspace(k_min, k_max, num_k)
max_iter = 500
tol = 1e-6
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 2/8
# Production function f(k, A)
def f(k, A):
return A * k**alpha + (1 - delta) * k
# Utility function u(c) = log(c)
def u(c):
return np.log(np.maximum(c, 1e-8)) # Avoid log(0) issues
# Consumption function ensuring positivity
def c(k, A, k_prime):
return np.maximum(f(k, A) - k_prime, 1e-8) # Ensure consumption is positive
# Marginal utility function u'(c) = 1/c
def mu_c(c):
return 1 / np.maximum(c, 1e-8) # Avoid division by zero
# First-order condition for root-finding
def bellman_foc(k_prime, k, A, V_prime, z):
return -mu_c(c(k, A, k_prime)) + beta * V_prime[z](k_prime)
# Howard's Policy Iteration
def policy_iteration(max_iter=500, tol=1e-6):
policy = np.full((num_k, 2), 0.5 * k_grid[:, None]) # Initial policy guess
V = np.zeros((num_k, 2)) # Initial value function
for iteration in range(max_iter):
# Step 1: Policy Evaluation
V_new = np.zeros_like(V)
V_splines = [CubicSpline(k_grid, V[:, z], bc_type='natural') for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
k_next = policy[i, z]
V_new[i, z] = u(c(k, A, k_next)) + beta * np.dot(pi, [V_splines[0](k_next), V_splines[1](k_next)])
# Check convergence
V_converged = np.max(np.abs(V_new - V)) < tol
V = V_new # Update value function
# Step 2: Policy Improvement
policy_stable = True
V_prime_splines = [CubicSpline(k_grid, np.gradient(V[:, z], k_grid), bc_type='natural') for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
solution = root(bellman_foc, x0=policy[i, z], args=(k, A, V_prime_splines, z))
k_opt = solution.x[0] if solution.success and solution.x[0] > 0 else max(1e-6, f(k, A) * 0.5)
if np.abs(k_opt - policy[i, z]) > tol:
policy_stable = False
policy[i, z] = k_opt
if policy_stable and V_converged:
break
return V, policy
# Compute solution
V_policy, policy_policy = policy_iteration()
# Plot Value Function
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_grid, V_policy[:, 0], 'b--', label='Value Function (Low A)')
plt.plot(k_grid, V_policy[:, 1], 'orange', label='Value Function (High A)')
plt.xlabel('Capital')
plt.ylabel('Value')
plt.legend()
plt.title('Value Function')
# Plot Policy Function
plt.subplot(1, 2, 2)
plt.plot(k_grid, policy_policy[:, 0], 'r--', label='Policy Function (Low A)')
plt.plot(k_grid, policy_policy[:, 1], 'b', label='Policy Function (High A)')
plt.xlabel('Capital Today')
plt.ylabel('Capital Tomorrow')
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 3/8
plt.legend()
plt.title('Policy Function')
plt.tight_layout()
plt.show()
# Compare steady-state values
k_star_numerical_low = policy_policy[np.argmin(np.abs(policy_policy[:, 0] - k_grid)), 0]
k_star_numerical_high = policy_policy[np.argmin(np.abs(policy_policy[:, 1] - k_grid)), 1]
k_star_analytical = ((1 - beta * (1 - delta)) / (beta * alpha)) ** (1 / (alpha - 1))
print(f"Analytical Steady-State Capital: {k_star_analytical:.4f}")
print(f"Numerical Steady-State Capital (Low A): {k_star_numerical_low:.4f}")
print(f"Numerical Steady-State Capital (High A): {k_star_numerical_high:.4f}")
Analytical Steady-State Capital: 2.6257
Numerical Steady-State Capital (Low A): 2.0869
Numerical Steady-State Capital (High A): 3.2820
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.interpolate import interp1d
# Parameters
alpha = 0.3 # Capital share in output
beta = 0.99 # Discount factor
delta = 0.1 # Depreciation rate
A_low, A_high = 0.9, 1.1 # Productivity states
pi = np.array([0.5, 0.5]) # Transition probabilities (i.i.d.)
k_min, k_max, num_k = 0.1, 10, 200 # Capital grid
k_grid = np.linspace(k_min, k_max, num_k)
max_iter = 500
tol = 1e-6
# Production function f(k, A)
def f(k, A):
return A * k**alpha + (1 - delta) * k
# Utility function u(c) = log(c)
def u(c):
return np.log(np.maximum(c, 1e-8)) # Avoid log(0) issues
# Consumption function ensuring positivity
def c(k, A, k_prime):
return np.maximum(f(k, A) - k_prime, 1e-8) # Ensure consumption is positive
# Marginal utility function u'(c) = 1/c
def mu_c(c):
return 1 / np.maximum(c, 1e-8) # Avoid division by zero
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 4/8
# First-order condition for root-finding
def bellman_foc(k_prime, k, A, V_prime, z):
return -mu_c(c(k, A, k_prime)) + beta * np.interp(k_prime, k_grid, V_prime[z])
# Howard's Policy Iteration (with Linear Interpolation)
def policy_iteration(max_iter=500, tol=1e-6):
policy = np.full((num_k, 2), 0.5 * k_grid[:, None]) # Initial policy guess
V = np.zeros((num_k, 2)) # Initial value function
for iteration in range(max_iter):
# Step 1: Policy Evaluation
V_new = np.zeros_like(V)
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
k_next = policy[i, z]
V_interp = [interp1d(k_grid, V[:, j], kind='linear', fill_value="extrapolate") for j in range(2)]
V_new[i, z] = u(c(k, A, k_next)) + beta * np.dot(pi, [V_interp[0](k_next), V_interp[1](k_next)])
# Check convergence
V_converged = np.max(np.abs(V_new - V)) < tol
V = V_new # Update value function
# Step 2: Policy Improvement
policy_stable = True
V_prime_grid = [np.gradient(V[:, z], k_grid) for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
solution = root(bellman_foc, x0=policy[i, z], args=(k, A, V_prime_grid, z))
k_opt = solution.x[0] if solution.success and solution.x[0] > 0 else max(1e-6, f(k, A) * 0.5)
if np.abs(k_opt - policy[i, z]) > tol:
policy_stable = False
policy[i, z] = k_opt
if policy_stable and V_converged:
break
return V, policy
# Compute solution
V_policy, policy_policy = policy_iteration()
# Plot Value Function
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_grid, V_policy[:, 0], 'b--', label='Value Function (Low A)')
plt.plot(k_grid, V_policy[:, 1], 'orange', label='Value Function (High A)')
plt.xlabel('Capital')
plt.ylabel('Value')
plt.legend()
plt.title('Value Function (Linear Interpolation)')
# Plot Policy Function
plt.subplot(1, 2, 2)
plt.plot(k_grid, policy_policy[:, 0], 'r--', label='Policy Function (Low A)')
plt.plot(k_grid, policy_policy[:, 1], 'b', label='Policy Function (High A)')
plt.xlabel('Capital Today')
plt.ylabel('Capital Tomorrow')
plt.legend()
plt.title('Policy Function (Linear Interpolation)')
plt.tight_layout()
plt.show()
# Compare steady-state values
k_star_numerical_low = policy_policy[np.argmin(np.abs(policy_policy[:, 0] - k_grid)), 0]
k_star_numerical_high = policy_policy[np.argmin(np.abs(policy_policy[:, 1] - k_grid)), 1]
k_star_analytical = ((1 - beta * (1 - delta)) / (beta * alpha)) ** (1 / (alpha - 1))
print(f"Analytical Steady-State Capital: {k_star_analytical:.4f}")
print(f"Numerical Steady-State Capital (Low A): {k_star_numerical_low:.4f}")
print(f"Numerical Steady-State Capital (High A): {k_star_numerical_high:.4f}")
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 5/8
Analytical Steady-State Capital: 4.1870
Numerical Steady-State Capital (Low A): 3.2839
Numerical Steady-State Capital (High A): 5.2251
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import root
from scipy.interpolate import interp1d
# Parameters
alpha = 0.3 # Capital share in output
beta = 0.99 # Discount factor
delta = 0.1 # Depreciation rate
A_low, A_high = 0.9, 1.1 # Productivity states
pi = np.array([0.5, 0.5]) # Transition probabilities (i.i.d.)
k_min, k_max, num_k = 0.1, 10, 200 # Capital grid
k_grid = np.linspace(k_min, k_max, num_k)
max_iter = 500
tol = 1e-6
# Production function f(k, A)
def f(k, A):
return A * k**alpha + (1 - delta) * k
# Utility function u(c) = log(c)
def u(c):
return np.log(np.maximum(c, 1e-8)) # Avoid log(0) issues
# Consumption function ensuring positivity
def c(k, A, k_prime):
return np.maximum(f(k, A) - k_prime, 1e-8) # Ensure consumption is positive
# Marginal utility function u'(c) = 1/c
def mu_c(c):
return 1 / np.maximum(c, 1e-8) # Avoid division by zero
# First-order condition for root-finding
def bellman_foc(k_prime, k, A, V_prime, z):
return -mu_c(c(k, A, k_prime)) + beta * np.interp(k_prime, k_grid, V_prime[z])
# Howard's Policy Iteration (with Linear Interpolation)
def policy_iteration(max_iter=500, tol=1e-6):
policy = np.full((num_k, 2), 0.5 * k_grid[:, None]) # Initial policy guess
V = np.zeros((num_k, 2)) # Initial value function
for iteration in range(max_iter):
# Step 1: Policy Evaluation
V_new = np.zeros_like(V)
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 6/8
k_next = policy[i, z]
V_interp = [interp1d(k_grid, V[:, j], kind='linear', fill_value="extrapolate") for j in range(2)]
V_new[i, z] = u(c(k, A, k_next)) + beta * np.dot(pi, [V_interp[0](k_next), V_interp[1](k_next)])
# Check convergence
V_converged = np.max(np.abs(V_new - V)) < tol
V = V_new # Update value function
# Step 2: Policy Improvement
policy_stable = True
V_prime_grid = [np.gradient(V[:, z], k_grid) for z in range(2)]
for i, k in enumerate(k_grid):
for z, A in enumerate([A_low, A_high]):
solution = root(bellman_foc, x0=policy[i, z], args=(k, A, V_prime_grid, z))
k_opt = solution.x[0] if solution.success and solution.x[0] > 0 else max(1e-6, f(k, A) * 0.5)
if np.abs(k_opt - policy[i, z]) > tol:
policy_stable = False
policy[i, z] = k_opt
if policy_stable and V_converged:
break
return V, policy
# Compute solution
V_policy, policy_policy = policy_iteration()
# Plot Value Function
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(k_grid, V_policy[:, 0], 'b--', label='Value Function (Low A)')
plt.plot(k_grid, V_policy[:, 1], 'orange', label='Value Function (High A)')
plt.xlabel('Capital')
plt.ylabel('Value')
plt.legend()
plt.title('Value Function (Linear Interpolation)')
# Plot Policy Function
plt.subplot(1, 2, 2)
plt.plot(k_grid, policy_policy[:, 0], 'r--', label='Policy Function (Low A)')
plt.plot(k_grid, policy_policy[:, 1], 'b', label='Policy Function (High A)')
plt.xlabel('Capital Today')
plt.ylabel('Capital Tomorrow')
plt.legend()
plt.title('Policy Function (Linear Interpolation)')
plt.tight_layout()
plt.show()
# Compare steady-state values
k_star_numerical_low = policy_policy[np.argmin(np.abs(policy_policy[:, 0] - k_grid)), 0]
k_star_numerical_high = policy_policy[np.argmin(np.abs(policy_policy[:, 1] - k_grid)), 1]
k_star_analytical = ((1 - beta * (1 - delta)) / (beta * alpha)) ** (1 / (alpha - 1))
print(f"Analytical Steady-State Capital: {k_star_analytical:.4f}")
print(f"Numerical Steady-State Capital (Low A): {k_star_numerical_low:.4f}")
print(f"Numerical Steady-State Capital (High A): {k_star_numerical_high:.4f}")
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 7/8
Analytical Steady-State Capital: 4.1870
Numerical Steady-State Capital (Low A): 3.2839
Numerical Steady-State Capital (High A): 5.2251
3/6/25, 9:49 AM Stochastic_Ramsey_Linear_vs_Cubic.ipynb - Colab
https://colab.research.google.com/drive/1yeB1kpezETqLK3mTYKFSF7LJCFKokLl6 8/8